In [17]:
import numpy as np
import pandas as pd
def show_matrix(matrix):
    for i in range(len(matrix)):
        print()
        for j in range(len(matrix[i])):
            print(matrix[i][j], end='\t\t\t')

In [18]:
def mse(point1, point2):
    return np.power(point1-point2, 2)

def eclidean_distance(point1, point2):
    return np.sqrt(np.sum(mse(point1, point2)))

def distance(cluster1,cluster2):
    return eclidean_distance(cluster1, cluster2)

def min_distance(arr1, arr2, X):
    min_dist = float('inf')
    
    for idx1 in arr1:
        for idx2 in arr2:
            dist = distance(X[idx1], X[idx2])
            if dist < min_dist:
                min_dist = dist
    
    return min_dist

def distance_mat(clusters, data):
    cluster_len = len(clusters)
    dist_mat = np.zeros((cluster_len, cluster_len))
    for i1 in range(cluster_len):
        for i2 in range(cluster_len):
            if i2 > i1:
                dist_mat[i1, i2] = min_distance(clusters[i1], clusters[i2], data)
            else : 
                dist_mat[i1, i2] = np.inf
    return dist_mat

def min_matrix(matrix):
    min_index = np.argmin(matrix)
    _, cols = matrix.shape
    i, j= min_index // cols, min_index % cols
    return i, j, matrix[i, j]
    
def linkage(clusters, data, num_clusters):
    linkage_matrix = []
    for i in range(num_clusters):
        linkage_matrix.append([])
        print(clusters)
        dist_mat = distance_mat(clusters, data)
        i_min, j_min, min_data = min_matrix(dist_mat)
        linkage_matrix[-1].append(clusters[i_min])
        linkage_matrix[-1].append(clusters[j_min])
        linkage_matrix[-1].append(min_data)
        clusters[i_min] = clusters[i_min] + clusters[j_min]
        clusters.pop(j_min)
        print(dist_mat)
    print()
    print(pd.DataFrame(linkage_matrix))
    return linkage_matrix


def compare_arr(arr1, arr2):
    for i in arr1:
        for j in arr2:
            if i == j:
                return True
    return False



def inconsistency(linkage_matrix):
    inconsistency_matrix = np.zeros(shape=(len(linkage_matrix), 4)).astype('float64')
    for i in range(len(linkage_matrix)):
        data = []
        for j in range(i,-1,-1):
            flags = np.array([
                compare_arr(linkage_matrix[i][0], linkage_matrix[j][0]),
                compare_arr(linkage_matrix[i][0], linkage_matrix[j][1]),
                compare_arr(linkage_matrix[i][1], linkage_matrix[j][0]),
                compare_arr(linkage_matrix[i][1], linkage_matrix[j][1])
                ])
            if flags.any() == True:
                    data.append(linkage_matrix[j][2])
        
        inconsistency_matrix[i][0] = np.mean(data)
        inconsistency_matrix[i][1] = np.std(data)
        inconsistency_matrix[i][2] = len(data)
        inconsistency_matrix[i][3] = 0 if inconsistency_matrix[i,1] == 0 else (linkage_matrix[i][2] - inconsistency_matrix[i,0]) / inconsistency_matrix[i,1]
    print()
    print(pd.DataFrame(inconsistency_matrix))
    # return inconsistency_matrix

In [19]:
from sklearn.datasets import make_blobs

num_clusters = 1
num_samples = 3
X, y = make_blobs(n_samples=num_samples, centers=num_clusters, cluster_std=2, random_state=42)
X += 10

In [20]:
# X = [
#     [4, 4, 1, 1],
#     [3, 0, 1, 1],
#     [8, 2, 1, 1],
#     [8, 0, 1, 1],
#     [8.5, 0, .5, .5]
# ]
np.random.seed(42)
X=np.random.normal(loc=0, scale=3, size=(10,2))
clusters = [
    [i] for i in range(10)
]
lm = linkage(clusters, np.asarray(X), 9)
print()
inconsistency(lm)

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
[[        inf  5.00442043  2.21138648  4.2342469   3.54589604  3.04331744
   5.37961161  6.78520445  4.72773132  5.68930514]
 [        inf         inf  5.89809489  3.59832646  4.45918899  6.8342883
  10.38053795  9.4763013   6.16168212  9.96633451]
 [        inf         inf         inf  6.21473934  2.43468846  0.97763805
   5.23601706  4.57936166  2.8571978   4.0718064 ]
 [        inf         inf         inf         inf  6.18297589  7.15802391
   8.98722683 10.68498786  7.89408863  9.92161594]
 [        inf         inf         inf         inf         inf  3.02492396
   7.67043954  5.01711448  1.76812588  6.01035463]
 [        inf         inf         inf         inf         inf         inf
   4.83080427  3.79557032  2.86216253  3.1373705 ]
 [        inf         inf         inf         inf         inf         inf
          inf  7.15850459  7.66990678  3.76311239]
 [        inf         inf         inf         inf         inf         inf
 